In [1]:
import pandas as pd
import requests
import json
from trading_ig import IGService
from trading_ig_config import config
from datetime import datetime, timedelta
import numpy as np
import talib
import igapi

sess = requests.Session()

In [2]:
try:
    # ig_service = IGService(config.username, config.password, config.api_key, config.acc_type)
    # ig_service.create_session(version='3')
    ig_service = igapi.IG(config.api_key, config.username, config.password, config.acc_type)
    print(ig_service.getVersion())
    ig_service.login()
    print("Login Successful")
except Exception as e:
    print(e)

account = ig_service.account()

pd.DataFrame(account)

1.0.6


Login Successful


,accountName,accountAlias,status,accountType,preferred,balance,currency,canTransferFrom,canTransferTo
accountId,,,,,,,,,
XQWAU,Demo-SpreadBet,None,ENABLED,SPREADBET,False,"{'balance': 79072.57, 'deposit': 0.0, 'profitL...",GBP,True,True
XQWAV,Demo-cfd,None,ENABLED,CFD,True,"{'balance': 8838.57, 'deposit': 0.0, 'profitLo...",GBP,True,True


In [3]:
account.loc['XQWAV', 'balance']['balance']

8838.57

In [4]:
price = ig_service.getPrice('CS.D.EURUSD.MINI.IP', 'DAY', 10)
print(price)

  marketStatus  netChange  percentageChange           updateTime  delayTime  \
0   EDITS_ONLY          0                 0  2024-04-21 23:05:00          0   

       bid    offer     high      low binaryOdds  decimalPlacesFactor  \
0  1.06559  1.06568  1.06568  1.06502       None                    5   

   scalingFactor  controlledRiskExtraSpread  
0          10000                          2  


In [5]:

def list_epics(term):
    if config.acc_type == 'LIVE':
        endpoint = 'https://api.ig.com/gateway/deal'
        response = sess.get(endpoint+'/markets'+f'?searchTerm={term}', headers={'X-IG-API-KEY': config.api_key, 
                        'CST': 'ffc8d032d461a40d429456316494b2fd90c893a4387f0f2275a02dd36dce03CC01113', 
                        'X-SECURITY-TOKEN': 'f0e2605109cbf6ee60fe13739cae92498965061502813699535eaa4ccc74b4CD01111', 
                        'Content-Type': 'application/json;', 'Accept': 'application/json; charset=UTF-8', 'VERSION': '1'})
        return response.json()

    elif config.acc_type == 'demo':
        endpoint = 'https://demo-api.ig.com/gateway/deal'
        response = sess.get(endpoint+'/markets'+f'?searchTerm={term}', headers={'X-IG-API-KEY': config.api_key, 
                        'CST': 'ffc8d032d461a40d429456316494b2fd90c893a4387f0f2275a02dd36dce03CC01113', 
                        'X-SECURITY-TOKEN': 'f0e2605109cbf6ee60fe13739cae92498965061502813699535eaa4ccc74b4CD01111', 
                        'Content-Type': 'application/json;', 'Accept': 'application/json; charset=UTF-8', 'VERSION': '1'})
        return response.json()
list_epics("FTSE")

In [6]:
end_date = datetime.now()
start_date = end_date - timedelta(weeks=12)
start=str(start_date)
end=str(end_date)
prices = ig_service.getPrices(epic='IX.D.FTSE.IFM.IP', 
                              resolution='DAY', 
                              start=start, 
                              end=end
                              )
prices

,snapshotTime,openPrice,closePrice,highPrice,lowPrice,lastTradedVolume
0,2024/01/29 00:00:00,"{'bid': 7638.3, 'ask': 7642.3, 'lastTraded': N...","{'bid': 7654.9, 'ask': 7658.9, 'lastTraded': N...","{'bid': 7665.0, 'ask': 7666.0, 'lastTraded': N...","{'bid': 7629.1, 'ask': 7630.9, 'lastTraded': N...",70687
1,2024/01/30 00:00:00,"{'bid': 7654.8, 'ask': 7658.8, 'lastTraded': N...","{'bid': 7659.7, 'ask': 7663.7, 'lastTraded': N...","{'bid': 7684.5, 'ask': 7685.5, 'lastTraded': N...","{'bid': 7641.1, 'ask': 7642.1, 'lastTraded': N...",71305
2,2024/01/31 00:00:00,"{'bid': 7659.8, 'ask': 7663.8, 'lastTraded': N...","{'bid': 7618.1, 'ask': 7622.1, 'lastTraded': N...","{'bid': 7689.5, 'ask': 7690.5, 'lastTraded': N...","{'bid': 7603.0, 'ask': 7605.0, 'lastTraded': N...",106530
3,2024/02/01 00:00:00,"{'bid': 7618.2, 'ask': 7622.2, 'lastTraded': N...","{'bid': 7659.3, 'ask': 7663.3, 'lastTraded': N...","{'bid': 7673.6, 'ask': 7674.6, 'lastTraded': N...","{'bid': 7598.8, 'ask': 7600.8, 'lastTraded': N...",101808
4,2024/02/02 00:00:00,"{'bid': 7659.2, 'ask': 7663.2, 'lastTraded': N...","{'bid': 7625.2, 'ask': 7629.2, 'lastTraded': N...","{'bid': 7684.0, 'ask': 7687.0, 'lastTraded': N...","{'bid': 7608.4, 'ask': 7609.4, 'lastTraded': N...",79200
...,...,...,...,...,...,...
65,2024/04/15 00:00:00,"{'bid': 7947.0, 'ask': 7951.0, 'lastTraded': N...","{'bid': 7915.8, 'ask': 7919.8, 'lastTraded': N...","{'bid': 8012.1, 'ask': 8013.1, 'lastTraded': N...","{'bid': 7900.6, 'ask': 7902.6, 'lastTraded': N...",95866
66,2024/04/16 00:00:00,"{'bid': 7915.9, 'ask': 7919.9, 'lastTraded': N...","{'bid': 7836.5, 'ask': 7840.5, 'lastTraded': N...","{'bid': 7915.9, 'ask': 7919.9, 'lastTraded': N...","{'bid': 7793.0, 'ask': 7794.0, 'lastTraded': N...",147189
67,2024/04/17 00:00:00,"{'bid': 7836.6, 'ask': 7840.6, 'lastTraded': N...","{'bid': 7869.8, 'ask': 7873.8, 'lastTraded': N...","{'bid': 7894.1, 'ask': 7895.1, 'lastTraded': N...","{'bid': 7792.7, 'ask': 7794.7, 'lastTraded': N...",113568
68,2024/04/18 00:00:00,"{'bid': 7869.9, 'ask': 7873.9, 'lastTraded': N...","{'bid': 7861.0, 'ask': 7865.0, 'lastTraded': N...","{'bid': 7898.4, 'ask': 7900.4, 'lastTraded': N...","{'bid': 7844.8, 'ask': 7845.8, 'lastTraded': N...",78334


In [7]:
bid_prices = [entry['bid'] for entry in prices['closePrice'].tolist()] 
bid_prices

[7654.9,
 7659.7,
 7618.1,
 7659.3,
 7625.2,
 7629.1,
 7624.6,
 7694.0,
 7638.5,
 7610.6,
 7589.6,
 7589.8,
 7581.7,
 7484.0,
 7562.8,
 7620.4,
 7693.8,
 7703.7,
 7723.4,
 7715.5,
 7657.3,
 7694.0,
 7707.5,
 7713.7,
 7681.8,
 7682.6,
 7606.1,
 7657.6,
 7679.6,
 7684.8,
 7638.2,
 7638.8,
 7644.2,
 7692.6,
 7651.3,
 7650.7,
 7700.9,
 7759.3,
 7776.6,
 7726.9,
 7735.8,
 7737.0,
 7712.6,
 7734.9,
 7773.1,
 7883.7,
 7924.4,
 7933.9,
 7908.0,
 7919.8,
 7937.9,
 7962.7,
 7992.4,
 7969.9,
 7937.6,
 7924.9,
 7895.5,
 7917.9,
 7919.2,
 7946.7,
 7945.5,
 7947.4,
 7947.0,
 7927.5,
 7946.9,
 7915.8,
 7836.5,
 7869.8,
 7861.0,
 7924.7]

In [14]:
ask_prices = [entry['ask'] for entry in prices['closePrice'].tolist()] 
# ask_prices

In [13]:
# Using the midpoint between bid and ask prices as the closing price
close = [(bid + ask) / 2 for bid, ask in zip(bid_prices, ask_prices)]
# close

In [10]:
high = max(max(bid_prices), max(ask_prices))
low = min(min(bid_prices), min(ask_prices))

In [11]:
print(high)
print(low)

7996.4
7484.0


In [36]:
class TradingBot:
    def __init__(self, symbol='IX.D.FTSE.IFM.IP'):
        self.symbol = symbol
        # self.account_info = ig_service.switch_account(config.acc_type, config.acc_id)
        # self.equity = self.account_info['availableToDeal']
        self.equity = account.loc['XQWAV', 'balance']['balance']
        self.risk_limit = 0.05 * self.equity  # Risk not more than 5% of equity

    # def initialize_account_info(self, default_account):
    #     try:
    #         return ig_service.switch_account(default_account)
    #     except Exception as e:
    #         print("Error initializing account info:", e)
    #         return None

    def fetch_historical_prices(self, interval='DAY'):
        try:
            end_date = datetime.now()
            start_date = end_date - timedelta(weeks=4)
            prices = ig_service.getPrices(
                epic=self.symbol,
                resolution=interval,
                start=str(start_date),
                end=str(end_date)
            )
            return prices
        except Exception as e:
            print("Error fetching historical prices:", e)
            return []

    def calculate_support_resistance(self, prices):
        try:
            bid_prices = [entry['bid'] for entry in prices['closePrice'].tolist()]
            ask_prices = [entry['ask'] for entry in prices['closePrice'].tolist()]
            high = max(max(bid_prices), max(ask_prices))
            low = min(min(bid_prices), min(ask_prices))

            # Support level is the lowest closing price
            support = low

            # Resistance level is the highest closing price
            resistance = high

            return support, resistance
        except Exception as e:
            print("Error calculating support and resistance levels:", e)
            return None, None

    def calculate_moving_average(self, prices, window=4):
        try:
            bid_prices = [entry['bid'] for entry in prices['closePrice'].tolist()]
            ask_prices = [entry['ask'] for entry in prices['closePrice'].tolist()]
            
            close_prices = np.array([(bid + ask) / 2 for bid, ask in zip(bid_prices, ask_prices)])
            # Calculate the simple moving average (SMA) using Ta-Lib
            moving_average = talib.SMA(close_prices, timeperiod=window)[-1]
            return moving_average
        except Exception as e:
            print("Error calculating moving average:", e)
            return None

    def calculate_pivot_point(self,prices):
        """
        Calculate the pivot point.

        Args:
        data (dict): Dictionary containing bid, ask, and last traded prices.

        Returns:
        float: The pivot point.
        """
        high = max([entry['ask'] for entry in prices['closePrice'].tolist()])
        low =  min([entry['bid'] for entry in prices['closePrice'].tolist()])
        close = max([entry['lastTraded'] if entry['lastTraded'] is not None else (entry['bid'] + entry['ask']) / 2 for entry in prices['closePrice'].tolist()])
        pivot_point = (high + low + close) / 3
        return pivot_point

    def generate_signal(self):
        try:
            prices = self.fetch_historical_prices()
            if prices is None or prices.empty:
                return 'hold'
            # else:
            #     return prices

            current_price = prices.iloc[-1]['closePrice']['bid']
            support_level, resistance_level = self.calculate_support_resistance(prices)
            moving_average = self.calculate_moving_average(prices)
            pivot_point = self.calculate_pivot_point(prices)

            if moving_average is None or pivot_point is None:
                return 'hold'
            # else:
                # return moving_average, pivot_point
            # return current_price, moving_average, support_level, resistance_level, pivot_point
            # Generating the signal based on conditions
            if current_price > moving_average and current_price > resistance_level and current_price > pivot_point:
                return 'buy'
            elif current_price < moving_average and current_price < support_level and current_price < pivot_point:
                return 'sell'
            else:
                return 'hold'
        except Exception as e:
            print("Error generating signal:", e)
        #     return 'hold'


    def execute_trade(self, size=1):
        try:
            side = self.generate_signal()
            if side == 'buy':
                ig_service.createPosition(
                    currency = 'GBP',
                    epic=self.symbol,
                    direction='BUY',
                    orderType='MARKET',
                    expiry='-',
                    size=size,
                    forceOpen=True,
                    limitDistance=None,
                    stopDistance=None,
                    guaranteed_stop=False
                )
                return f"Buy trade executed for {size} units of {self.symbol}"
            elif side == 'sell':
                ig_service.createPosition(
                    currency = 'GBP',
                    epic=self.symbol,
                    direction='SELL',
                    orderType='MARKET',
                    expiry='-',
                    size=size,
                    forceOpen=True,
                    limitDistance=None,
                    stopDistance=None,
                    guaranteed_stop=False
                )
                return f"Sell trade executed for {size} units of {self.symbol}"
            else:
                return "No trade executed."
        except Exception as e:
            print("Error executing trade:", e)
            return "Error executing trade"


In [37]:
trading_bot = TradingBot()
prices = trading_bot.fetch_historical_prices()
# trading_bot.calculate_support_resistance(prices)
# trading_bot.calculate_moving_average(prices)
# trading_bot.calculate_pivot_point(prices)
# trading_bot.generate_signal()
trading_bot.execute_trade()

'No trade executed.'